## TODOs
* in compose_tfms2 return a custom type
* this custom type should handle ToTensor

In [ ]:
from fastai2.basics import *
from fastai2.text.all import *
from pigboat.text.all import *

In [ ]:
from snorkel.labeling import model

In [ ]:
source = untar_data(URLs.IMDB_SAMPLE)

In [ ]:
df = pd.read_csv(source/'texts.csv')

In [ ]:
splits = ColSplitter('is_valid')(df)

In [ ]:
df_train = df.iloc[splits[0]]
df_valid = df.iloc[splits[1]]

In [ ]:
POS,NEG = 'positive','negative'
vocab = ABSTAIN,POS,NEG

In [ ]:
POS_WORDS  = ['enjoy', 'liked', 'love', 'masterpiece', 'enjoyed', 'loved', 'nice', 'good acting']
NEG_WORDS = ['hate', 'poor', 'bad', 'hated', 'dislike', 'disliked', 'suck', 'lame', 'worst']

In [ ]:
lblr = []
lblr += [WordLabeller(w, POS) for w in POS_WORDS]
lblr += [WordLabeller(w, NEG) for w in NEG_WORDS]

In [ ]:
pipe = Pipeline2([Tokenizer.from_df('text')], lblr)

In [ ]:
get_x = [ColReader('text'), pipe, Categorize2(vocab=vocab)]
get_y = [ColReader('label'), Categorize(vocab=vocab)]
dset_train = Datasets(df, [get_x], splits=splits).train
dset_valid = Datasets(df, [get_x, get_y], splits=splits, do_setup=False).valid

NameError: name 'Categorize2' is not defined

In [ ]:
dls = DataLoaders.from_dsets(dset_train, dset_valid, shuffle=False, device=default_device(), drop_last=False)

In [ ]:
dset = Datasets(df_train, [[ColReader('text'), pipe]])
dls2 = dset.dataloaders(shuffle_train=False)

In [ ]:
x1 = dls.one_batch()[0]
x2 = dls2.one_batch()[0]

In [ ]:
m = MajorityLabelVoter()

In [ ]:
learn = Learner(dls, m, loss_func=noop)

In [ ]:
preds,_ = learn.get_preds(ds_idx=0)

In [ ]:
idxs = (preds!=0).nonzero()

In [ ]:
(preds==lbls_true).float().mean()

tensor(0.4630)

In [ ]:
(preds[idxs]==lbls_true[idxs]).float().mean()

tensor(0.7080)

In [ ]:
# How to apply in parellel
task_lbls = torch.stack([o.data for o in tls])

In [ ]:
# lbls[lbls==vocab.o2i[ABSTAIN]] = -1
# Only works because ABSTAIN is already 0
task_lbls -= 1

In [ ]:
task_lbls_arr = np.array(task_lbls)

In [ ]:
# lbl_m = LabelModel(2, verbose=True)
lbl_m = MajorityLabelVoter()
# lbl_m.fit(task_lbls_arr, n_epochs=500, log_freq=100)

In [ ]:
lbl_m.score(task_lbls_arr, lbls_true.numpy()-1)

{'accuracy': 0.7165775401069518}

In [ ]:
dls = tls.dataloaders(shuffe=False)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b)

3

In [ ]:
ToTensor()(items[0])

TensorCategory(0)